# Parameter fitting for Markov random fields

In the following toy example we show how one can fit paramaters of the Markov fields and how the result is connected to fitting the model for conditional probability. 
Our toy model is a chain of four nodes and two parameters $\alpha$ and $\delta$ depicted below.

<img src = '../illustrations/markov-random-field-1d.png' width=60%>



## I. Direct analytical derivation 

As usual we construct the potential energy function directly.
* The deviations from zero are penalised by $\frac{1}{2}\cdot\delta^2 x_{i}^2$.
* The differeces between consecutive nodes are penalised by $\frac{1}{2}\cdot\alpha (x_{i}-x_{i+1})^2$.

The shape of the energy function guarantees that the resulting unnormalised distribution 

\begin{align*}
p[x_0,x_1,x_2,x_3]\propto\exp\left(-\frac{\delta^2}{2}\cdot \sum_{i=0}^3 x_i^2 -\frac{\alpha}{2}\cdot\sum_{i=0}^2(x_i-x_{i+1})^2\right)
\end{align*}

is a multivariate normal distribution with the inverse covariance matrix

\begin{align*}
\boldsymbol{\Sigma}^{-1}=
\begin{pmatrix}
\delta^2+\alpha & -\alpha          & 0                & 0\\
-\alpha         & \delta^2+2\alpha & -\alpha          & 0\\
0               & -\alpha          & \delta^2+2\alpha & 0\\
0               & 0                & -\alpha          & \delta^2+\alpha
\end{pmatrix}
\end{align*}

As the density of the multivariate normal distribution can be expressed in terms of inverse covariance matrix 

\begin{align*}
p[\boldsymbol{x}]=\frac{1}{(2\pi)^2}\cdot \sqrt{\det \boldsymbol{\Sigma}^{-1}}\cdot \exp\left(-\frac{1}{2}\cdot \boldsymbol{x}^T \boldsymbol{\Sigma}^{-1}\boldsymbol{x}\right)
\end{align*}

we can find normalised density without integrating as 


\begin{align*}
\det \boldsymbol{\Sigma}^{-1}=4\alpha^3 \delta^2 + 10\alpha^2\delta^4 + 6\alpha\delta^6 + \delta^8\enspace.
\end{align*}

Consequently, we can find coefficient by looking at the partial derivatives of the log-likelihood

\begin{align*}
\log p[\boldsymbol{x}] = const + \frac{1}{2} \cdot (4\alpha^3 \delta^2 + 10\alpha^2\delta^4 + 6\alpha\delta^6 + \delta^8) -\frac{\delta^2}{2}\cdot \sum_{i=0}^3 x_i^2 -\frac{\alpha}{2}\cdot\sum_{i=0}^2(x_i-x_{i+1})^2
\end{align*}

Hence we get equations

\begin{align*}
\frac{\partial \log p[\boldsymbol{x}]}{\partial \alpha}&=
6\alpha^2\delta^2 + 10\alpha\delta^4 + 3\delta^6 -\frac{1}{2}\cdot\sum_{i=0}^2(x_i-x_{i+1})^2=0\\
\frac{\partial \log p[\boldsymbol{x}]}{\partial \alpha}&=
4\alpha^3\delta + 20\alpha^2\delta^3 + 18\alpha\delta^5 + 4\delta^7
-\delta\cdot\sum_{i=0}^3 x_i^2=0
\end{align*}

that can be solved for $\alpha$ and $\delta$ but not in closed form.

**Alternative numerical optimisation  task.**
As we can find optimal parameters by maximising the log-likelihood directly which can be restated in terms of potential energy minimisation 

\begin{align*}
\Psi-\frac{1}{2}\cdot\ln \bigl(\det \boldsymbol{\Sigma}^{-1}\bigr)\to \min
\end{align*}

As all functions are computable and derivable, we can find $\alpha$ and $\delta$ with gradient decent any other optimisation technique.  
In principle, a similar formulae holds for any potential function

\begin{align*}
\Psi(\boldsymbol{x},\boldsymbol{\omega}) + \ln Z(\boldsymbol{\omega})\to \min
\end{align*}

However, for most potential functions we just do not know the normalising constant $Z(\boldsymbol{\omega})$. 


### Validation of formulae using Sympy

* For obvious reasons one does not want to derive the formulae manually. 
* We use `sympy` package for this but there are many other computer algebra systems 

In [1]:
import sympy as sp
from sympy import Matrix
from sympy import Symbol
from IPython.display import Math

In [2]:
# Introduce variables
x0 = Symbol('x_0')
x1 = Symbol('x_1')
x2 = Symbol('x_2')
x3 = Symbol('x_3')

a = Symbol('alpha')
d = Symbol('delta')

In [3]:
# Direct formula for the potential function
phi_1=d**2/2*(x0**2 + x1**2 + x2**2 + x3**2) + a/2*((x0-x1)**2+(x1-x2)**2+(x2-x3)**2) 
display(Math(f'\Psi_1={sp.latex(phi_1)}'))

# Potential function as a quadratic form
A = Matrix([
    [d*d+a, -a,      0,       0],
    [-a,    d*d+2*a, -a,      0],
    [0,     -a,      d*d+2*a, -a ],
    [0,     0,       -a,      d*d+a]
])
X = Matrix([x0, x1, x2, x3])
phi_2 = (X.T * A * X/2)[0]

display(Math(f'A={sp.latex(A)}'))
display(Math(f'\Psi_2=\\frac{1}{2}\cdot \\boldsymbol{{x}}^TA\\boldsymbol{{x}}={sp.latex(sp.expand(phi_2))}'))

# Check that both formulae are equal
display(Math(f'\Psi_1-\Psi_2={sp.expand(phi_1-phi_2)}'))

## Determinant of A and its derivatives
display(Math(f'\\det \\boldsymbol{{\Sigma}}^{{-1}}={sp.latex(A.det())}'))
display(Math(f'\\frac{{\\partial}}{{\\partial{sp.latex(a)}}}\det \\boldsymbol{{\Sigma}}^{{-1}}={sp.latex(sp.diff(A.det(),a))}'))
display(Math(f'\\frac{{\\partial}}{{\\partial{sp.latex(d)}}}\det \\boldsymbol{{\Sigma}}^{{-1}}={sp.latex(sp.diff(A.det(),d))}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## II. Conditional distributions

Let us now express conditional distribution for $x_1$ and $x_2$ in terms of $\alpha$ and $\delta$.
For that we can use a trivial observation 

\begin{align*}
p[x_1|x_0,x_2]=p[x_1|x_0,x_2,x_3]\propto p[x_0,\ldots,x_3]\\
p[x_2|x_1,x_3]=p[x_2|x_0,x_1,x_3]\propto p[x_0,\ldots,x_3]
\end{align*}

where the proportionality is wrt $x_1$ and $x_2$. 
Thus, we get

\begin{align*}
p[x_1|x_0,x_2]&\propto\exp\left(-\frac{(\delta^2+2\alpha) x_1^2-2\alpha x_1x_0-2\alpha x_1x_2}{2}\right)
\propto\exp\left(-\frac{ \left(x_1-\frac{\alpha (x_0+x_2)}{\delta^2+2\alpha}\right)^2}{\frac{2}{\delta^2+2\alpha}}\right)
\\
p[x_2|x_1,x_3]&\propto\exp\left(-\frac{(\delta^2+2\alpha) x_2^2-2\alpha x_1x_2-2\alpha x_2x_3}{2}\right)
\propto\exp\left(-\frac{ \left(x_2-\frac{\alpha (x_1+x_3)}{\delta^2+2\alpha}\right)^2}{\frac{2}{\delta^2+2\alpha}}\right)
\end{align*}

which again indicates that the conditional distributions are normal distributions.
As the parameters appear in both expressions we should try to maximise the product of conditional probabilities as a result, we have to solve a linear regression task. However, due to the constraints placed by the density function we have to solve the linear regrsttion is in the form $x_{i} = \gamma(x_{i-1}+x_{i+1})+\varepsilon$. 

In [8]:
display(sp.collect(sp.expand(2*phi_2), x1))
display(sp.collect(sp.expand(2*phi_2), x2))

alpha*x_0**2 + 2*alpha*x_2**2 - 2*alpha*x_2*x_3 + alpha*x_3**2 + delta**2*x_0**2 + delta**2*x_2**2 + delta**2*x_3**2 + x_1**2*(2*alpha + delta**2) + x_1*(-2*alpha*x_0 - 2*alpha*x_2)

alpha*x_0**2 - 2*alpha*x_0*x_1 + 2*alpha*x_1**2 + alpha*x_3**2 + delta**2*x_0**2 + delta**2*x_1**2 + delta**2*x_3**2 + x_2**2*(2*alpha + delta**2) + x_2*(-2*alpha*x_1 - 2*alpha*x_3)

## III. Density from conditional distribution

The previous derivation showed that if we build the potential energy function as a quadratic form from few smoothness constraints, the conditional distribution will be a normal distribution and the resulting linear regression task is symmetric. The latter is not surprising as the original density function is symmetric. 

Let us now condider the reverse task when we fix the conditional distribution 

\begin{align*}
 x_{i}=w_1 x_{i-1} + w_2 x_{i+1}+\varepsilon_i, \quad \varepsilon_i\sim\mathcal{N}(0,\sigma) 
\end{align*}

and seek for the full distribution. According to the celebrated Hammersley-Clifford theorem, the density function can be expressed

\begin{align*}
p[\boldsymbol{x}]\propto \exp\Bigl(-\Psi_1(x_0,x_1)-\Psi_2(x_1, x_2)-\Psi_3(x_2,x_3)\Bigr)\enspace.
\end{align*}

As a result we get two ways to express the conditional propability. From the construction we can conclude

\begin{align*}
p[x_i|x_{i-1},x_{i+1}]\propto \exp\left(-\frac{(x_i-w_1x_{i-1}-w_2x_{i+1})}{2\sigma^2}\right)\enspace.
\end{align*}

On the other hand, Bayes formula gives

\begin{align*}
p[x_i|x_{i-1},x_{i+1}]\propto p[\boldsymbol{x}]\propto\exp\Bigl(-\Psi_i(x_{i-1}, x_{i})-\Psi_{i+1}(x_i,x_{i+1})\Bigr)\enspace.
\end{align*}

Thus we get two equalities

\begin{align*}
(x_1-w_1x_0-w_2x_2)^2+c_1(x_0,x_2)&=2\sigma^2 (\Psi_1(x_0,x_1)+\Psi_2(x_1,x_2))\\
(x_2-w_1x_1-w_2x_3)^2+c_2(x_1,x_3)&=2\sigma^2 (\Psi_2(x_1,x_2)+\Psi_3(x_2,x_3))\\
\end{align*}

where $c_1$ does not depend on $x_1$ and $c_2$ does not depend on $x_2$. We can simplify this further by pushing terms under $c_1(x_0,x_2)$ and $c_2(x_1,x_3)$:

\begin{align*}
-2w_{1}x_{0}x_{1} - 2w_{2}x_{1}x_{2} + x_{1}^{2} +c_1(x_0,x_2)&=2\sigma^2 (\Psi_1(x_0,x_1)+\Psi_2(x_1,x_2))\\
-2w_{1}x_{1}x_{2} - 2w_{2}x_{2}x_{3} + x_{2}^{2} + c_2(x_1,x_3)&=2\sigma^2 (\Psi_2(x_1,x_2)+\Psi_3(x_2,x_3))\\
\end{align*}

From these equations we get the following constraints

\begin{align*}
\Psi_1(x_0,x_1)&= \frac{-2w_{1}x_{0}x_{1}}{\sigma^2} + g_{10}(x_0) + g_{11}(x_1) \\
\Psi_2(x_1,x_2)&= \frac{-2w_{2}x_{1}x_{2}}{\sigma^2} + g_{20}(x_1) + g_{21}(x_2)\\
\Psi_2(x_1,x_2)&= \frac{-2w_{1}x_{1}x_{2}}{\sigma^2} + \bar{g}_{20}(x_1) +\bar{g}_{21}(x_2)\\
\Psi_3(x_2,x_3)&= \frac{-2w_{2}x_{2}x_{3}}{\sigma^2} + g_{30}(x_2) + g_{31}(x_3)\\
\end{align*}

For obvious reasons these constraints can be satisfied only if $w_1=w_2=w$.
Under this assumption it is straightforward to come up with a symmetric subpotentials

\begin{align*}
\Psi_1(x_0,x_1)&= \frac{w(x_0-x_1)^2}{2\sigma^2} + \frac{(1-w)x_0^2}{2\sigma^2} 
+\frac{(1-2w)x_1^2}{4\sigma^2} \\
\Psi_2(x_1,x_2)&= \frac{w(x_1-x_2)^2}{2\sigma^2} + \frac{(1-2w)x_1^2}{4\sigma^2} 
+\frac{(1-2w)x_2^2}{4\sigma^2}\\
\Psi_3(x_2,x_3)&= \frac{w(x_2-x_3)^2}{2\sigma^2} + \frac{(1-2w)x_2^2}{4\sigma^2} 
+\frac{(1-w)x_3^2}{2\sigma^2}\\
\end{align*}

which gives the same potential function we built from smoothness constraints.
Of course this result is expected.

In [59]:
w = Symbol('w')
display(sp.expand(w/sigma**2*(x0-x1)**2/2 + (1-w)/sigma**2/2*x0**2 + (1/4/sigma**2-w/sigma**2/2)*x1**2))
display(sp.expand(w/sigma**2*(x1-x2)**2/2 + (1/4/sigma**2-w/sigma**2/2)*x1**2 + (1/4/sigma**2-w/sigma**2/2)*x2**2))
display(sp.expand(w/sigma**2*(x2-x3)**2/2 + (1/4/sigma**2-w/sigma**2/2)*x2**2 + (1/2/sigma**2-w/sigma**2/2)*x3**2))

-w*x_0*x_1/sigma**2 + x_0**2/(2*sigma**2) + 0.25*x_1**2/sigma**2

-w*x_1*x_2/sigma**2 + 0.25*x_1**2/sigma**2 + 0.25*x_2**2/sigma**2

-w*x_2*x_3/sigma**2 + 0.25*x_2**2/sigma**2 + 0.5*x_3**2/sigma**2